# Re-create OpenAI's Deep Research using OpenAI Agents SDK

### Four different agents will be made:

1. Searcher Agent

    - Takes search terms from *Planner agent* and then searches online for those terms
    - Uses `WebSearchTool` (one of OpenAI Agents SDK's *hosted tools*)

2. Planner Agent

    - Takes a general question from a user and then forms suitable searches based on it

3. Writer Agent

    - Takes data gathered from searches and then turns it into a summary report

4. Pusher Agent

    - Sends a notification to the user's phone with the summary

In [69]:
# Import packages and modules
import os
import requests
import asyncio

from dotenv import load_dotenv
from agents import Agent, trace, Runner, WebSearchTool, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from IPython.display import display, Markdown
from pprint import pprint

In [ ]:
# Initialize Pushover constants
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [71]:
# Load the OpenAI API key
load_dotenv(override=True)

True

## Searcher agent

When given a search term, search the internet for it, and finally return a summary of the results.

In [72]:
# System prompt
INSTRUCTIONS = "You're a research assistant. When given a search term, search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words long. Capture the main points. Write succinctly, no need to do complete sentences or good \
grammar. The summary will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Don't include any additional commentary other than the summary itself."

# Create the searcher agent
# Arm it with `WebSearchTool`
searcher_agent = Agent(
    name="SearcherAgent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4.1-mini",
    # Make sure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required"),
)

In [73]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025?"

# Set up tracing
with trace("Search"):
    # Run the searcher agent
    result = await Runner.run(searcher_agent, message)

display(Markdown(result.final_output))

As of November 2025, several AI agent frameworks have gained prominence for their capabilities and adoption:

- **LangChain**: A versatile framework for building context-aware, reasoning applications, offering extensive integrations and reusable components. ([linkedin.com](https://www.linkedin.com/pulse/ai-agent-frameworks-trends-popularity-2025-sachin-tiwari-c5tuc?utm_source=openai))

- **LangGraph**: An extension of LangChain, providing a graph-based orchestration layer for managing long-running, stateful agents with complex branching. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **AutoGen**: Microsoft's framework tailored for multi-agent collaboration and asynchronous task execution with human-in-the-loop oversight. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **CrewAI**: A rapidly growing framework focusing on multi-agent collaboration and workflow design, known for its ease of use and suitability for prototyping. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **Semantic Kernel**: Microsoft's open-source development kit for building enterprise-grade generative AI applications. ([medium.com](https://medium.com/%40dev-infinity101/top-5-frameworks-for-building-ai-agents-in-2025-ddacc314f19f?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework released in March 2025, focusing on creating multi-agent workflows with comprehensive tracing and guardrails. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: Announced in April 2025, this modular framework integrates with Google's ecosystem, supporting hierarchical agent compositions with minimal code. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, capable of independent reasoning, dynamic planning, and decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

- **$Agent^2$**: A novel agent-generates-agent framework for reinforcement learning automation, enabling intelligent LLM-driven generation of RL agents. ([arxiv.org](https://arxiv.org/abs/2509.13368?utm_source=openai))

- **AutoAgent**: A fully-automated, zero-code framework for LLM agents, allowing users to create and deploy agents through natural language alone. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

- **Agent Lightning**: A flexible and extensible framework enabling reinforcement learning-based training of LLMs for any AI agent, with decoupled agent execution and training. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

These frameworks are shaping the AI agent landscape, offering diverse tools for various applications. 

### Go look at the trace

https://platform.openai.com/traces

## Planner agent

When given a query, come up with a set of ideas for web searches that could be run.

In [74]:
NUM_SEARCHES = 5

# System prompt
INSTRUCTIONS = f"You're a research assistant. When given a query, come up with a set of web searches \
to perform to best answer the query. Output {NUM_SEARCHES} terms to query for."

# Use Pydantic objects to describe the Schema of the output
# This is sometimes called 'structured outputs'

# A web search
class WebSearchItem(BaseModel):
    # We want the agent to tell us why a search is important,
    # this way we get better outcomes

    # This field forces the agent to operate in a reasoning mode
    # and tell you why it's doing what it's doing
    # before telling you the answer
    # This biases it to be more likely to output tokens
    # consistent with its reasoning
    reason: str
    "Your reasoning for why this search is important to the query"

    # The actual search term
    query: str
    "The search term to use for the web search"

# A list of web searches to perform (a plan)
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    "A list of web searches to perform to best answer the query"

# Create the planner agent
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4.1-mini",
    # Passing the plan here ensures the output follows the schema
    output_type=WebSearchPlan,
)

In [75]:
# User query
message = "What are the most popular and successful AI Agent frameworks in November 2025?"

# Set up tracing
with trace("Search"):
    # Run the planner agent
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

# Now the agent presents us five queries with reasoning behind them

WebSearchPlan(searches=[WebSearchItem(reason='To find up-to-date information on popular AI agent frameworks as of November 2025', query='most popular AI agent frameworks November 2025'), WebSearchItem(reason='To identify successful AI agent platforms and their usage statistics in late 2025', query='successful AI agent platforms usage statistics 2025'), WebSearchItem(reason='To discover new and emerging AI agent frameworks launched or gaining traction in 2025', query='new AI agent frameworks 2025'), WebSearchItem(reason='To get expert reviews and comparisons of AI agent frameworks in 2025', query='AI agent framework reviews 2025'), WebSearchItem(reason='To check developer community and industry adoption trends for AI agent frameworks in 2025', query='AI agent frameworks developer adoption 2025')])


## Writer agent

Take the results of the internet searches and write a report.

In [76]:
# System prompt
INSTRUCTIONS = "You're a senior researcher tasked with writing a cohesive report for a research query. \
You'll be provided with the original query, and some initial research done by a research assistant.\
You should first come up with an outline for the report that describes the structure and \
flow of the report. Then, generate the report and return that as your final output.\
The final output should be in markdown format, and it should be lengthy and detailed. Aim \
for 5-10 pages of content, at least 1000 words."

# Use structured outputs again

class ReportData(BaseModel):
    short_summary: str
    "A short 2-3 sentence summary of the findings"

    markdown_report: str
    "The final report"

    follow_up_questions: list[str]
    "Suggested topics to research further"

# Create the writer agent
report_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    # Passing the structure here ensures the output follows the schema
    output_type=ReportData,
)

## Pusher agent

Send a push notification to the user's phone with the summary.

In [ ]:
# This is called a decoration
# Makes 'push' a tool instead of a function
@function_tool
# Uses Pushover,
# which can be used to send push notifications to a phone
# https://pushover.net
def push(message: str):
    "Send a push notification with this brief message"
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    return {"status": "success"}

In [78]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x105fd71a0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [79]:
# System prompt
INSTRUCTIONS = "You're a member of a research team and will be provided with a short summary of a report. \
When you receive the report summary, you send a push notification to the user using your tool, informing them that the research is complete, \
and including the report summary you receive."

# Create the pusher agent
pusher_agent = Agent(
    name="PusherAgent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4.1-mini",
    # Make sure the agent uses the tool
    model_settings=ModelSettings(tool_choice="required")
)